In [3]:
import logging
import os, sys
from pathlib import Path
import numpy as np

verbose = True

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import cv2


__file__ = os.path.abspath('helper_precomputing_CNN_features.ipynb')
DNN_lib_path = Path(__file__).parents[1].__str__()

# from contextlib import redirect_stdout
sys.path.append('..')

#from keras.preprocessing.image import img_to_array

# setup paths
path_cache = DNN_lib_path + '/cache/'
path_data = DNN_lib_path + '/data_cnn_ts_3d/'

# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s, [%(levelname)-8s] [%(filename)s:%(lineno)d] %(message)s',
    handlers=[
        logging.FileHandler(f"{DNN_lib_path}/logs.log"),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()

resnet_len_features = 2048
resnet50_model_size = (32, 32)


custom_model_name = 'ResNet50_test'
pretrained_model_name = 'ResNet50'
pooling = 'max'
""" 
Save pretrained features array computed over all frames of each video 
using given pretrained model and pooling method

:pretrained_model_name: pretrained model object loaded using `load_pretrained_model`
:pooling: pooling method used with pretrained model
:model_weights_path: path to custom model weights if we want to load CNN model we've fine-tuned to produce features (e.g. for LRCNN)
:custom_model_name: custom output name to append to pretrained model name

"""

# setup path to save features
path_features = path_cache + 'features/' + custom_model_name

if not os.path.exists(path_features):
    os.mkdir(path_features)

model = ResNet50(include_top=False, weights=None,
                 pooling=pooling, input_shape=(32, 32, 3))

bes_names = [be for be in os.listdir(path_data) if os.path.isdir(f"{path_data}/{be}")]

for be_name in bes_names:
    # build output path
    path_output = f"{path_features}/{be_name}"
    path_frames = f"{path_data}/{be_name}"

    if os.path.exists(path_output + '.npy'):
        if verbose:
            logger.info("Features already cached: {}.npy".format(path_output))
        continue

    # initialize features list
    features = []

    frame_paths = [
        f"{path_frames}/{frame}" for frame in os.listdir(path_frames)]

    # sort paths in sequence (they were created with incrementing filenames through time)
    frame_paths.sort()

    # load each frame in vid and get features
    for j, frame_path in enumerate(frame_paths):

        frame = np.load(frame_path, allow_pickle=True)
        frame[np.isnan(frame)] = 0
        frame = np.resize(frame, new_shape=(*resnet50_model_size, 3))
        print(f'shape: {np.shape(frame)}')
        frame = np.expand_dims(frame, axis=0)

        # get features from pretrained model
        feature = model.predict(frame).ravel()
        features.append(feature)

    # convert to arrays
    features = np.array(features)

    np.save(path_output, features)

shape: (32, 32, 3)
1/1 [==============================] - 1s 783ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 30ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 28ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 26ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 31ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 28ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 27ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 27ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 30ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 28ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 28ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 32ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 27ms/step
shape: (32, 32, 3)
1/1 [==============================] - 0s 30ms/step
shape

In [4]:
frame_paths = [f"{path_frames}/{frame}" for frame in os.listdir(path_frames)]
frames = [np.load(path_frame) for path_frame in frame_paths]

In [5]:
frames[0]

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])